In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

In [3]:
cols_drop = ['country_name','lat-lon','properati_url','image_thumbnail','title','extra','surface_in_m2']
FILE_TRAIN = 'data/propiedades_completo.csv'
FILE_TEST = 'data/properati_dataset_testing_noprice.csv'

# Training Dataset

In [4]:
propiedades = pd.read_csv(FILE_TRAIN, low_memory=False)

In [5]:
propiedades.shape

(338158, 29)

Las siguentes lineas se encargan de filtrar las propiedades no deseadas dentro del total de las propiedades a gran escala, esto siendo filtrando la zona en la cual se encuentran, el año de creacion del aviso, y valores sin sentido de precio. Como tambien, en el bloque siguente a este, se filtran toda propiedad que no se _venda_ en _dolares_.

In [6]:
propiedades = propiedades.loc[propiedades.state_name.str.contains('G.B.A') | propiedades.state_name.str.contains('Capital Federal')]
propiedades = propiedades.loc[~ propiedades.property_type.str.contains('store')]
propiedades['created_on'] = pd.to_datetime(propiedades['created_on'])
propiedades.insert(3, 'year', propiedades['created_on'].dt.year)
propiedades['year'] = propiedades['year'].fillna(0).astype(int)
propiedades = propiedades.loc[propiedades.year > 2012]
propiedades = propiedades.loc[propiedades.price < 20000000]
propiedades = propiedades.loc[propiedades.price > 10000]
propiedades = propiedades.drop(propiedades[cols_drop], axis=1)
propiedades.shape

(192820, 23)

In [7]:
propiedades.dropna(axis=0, how='any', subset=['currency'], inplace=True)
propiedades = propiedades.loc[propiedades.currency.str.contains('USD')]
propiedades = propiedades.loc[propiedades.operation.str.contains('sell')]


Dado a que avisos en en el set de prueba contenienan tipos de valores distintos a _int_ como valor de la descripcion, se los deja lo mas limpio de caracteres posible para luego, tratar de convertirlos en a _int_, de no ser posible, se lo deja como __NaN__

In [8]:
def validarExpensas(row):
    if(type(row['expenses']) == type(1) or type(row['expenses']) == type(1.)):
        return float(row['expenses'])
    if row['expenses'].isdigit():
        return float(row['expenses'])
    return np.nan

In [9]:
propiedades['expenses'] = propiedades['expenses'].replace('[\$,)]','',  \
        regex=True).replace('[(]','-', regex=True)
propiedades['expenses'] = propiedades.apply(lambda row: validarExpensas(row), axis=1)

Para poder utilizar los algoritmos de machine learning es necesario que todos los valores que se les da para procesar sean numericos, entonces se remplaza los valores referidos a la zona y tipo de propiedad con valores categoricos, y se termina de descarta todas las columnas que no son necesarias.

In [10]:
cols = ['created_on', 'year', 'currency', 'geonames_id', 'operation', 'price_aprox_local_currency', 'price_aprox_usd', 'price_per_m2', 'price_usd_per_m2']
propiedades.dropna(axis=0, how='any', subset=['price'], inplace=True)
propiedades = propiedades.drop(propiedades[cols], axis=1)

In [11]:
propiedades_categorias = propiedades
categories = ['place_with_parent_names','place_name','property_type','state_name']
dic_categories = {}    
for cat in categories:
    l = list(propiedades_categorias.groupby([cat]).count().transpose())
    dic_categories[cat] = l

def categorical(x, cat):
    try:
        return dic_categories[cat].index(x)
    except ValueError:
        return -1
    
dic_categories['property_type']

['PH', 'apartment', 'house']

In [12]:
for cat in categories:
    propiedades[cat] = propiedades[cat].apply(lambda x: categorical(x,cat))

In [13]:
CSV_NAME = 'data/training.csv'
propiedades.to_csv(CSV_NAME, index=False)

# Kaggle Test Dataset

Al igual que para los datos de entrenamiento, se transforman algunos valores ilogicos a __NaN__ para luego remplazarlos con valores mas comunes.
Cuando se combierten los valores categoricos a numeros, estas categorias se remplazan en base a las creadas con anterioridad para mantener cuerencia en los resultados.

In [24]:
propiedades = pd.read_csv(FILE_TEST, low_memory=False)

In [25]:
cols_drop = ['created_on','operation','country_name','lat-lon']

In [26]:
propiedades['surface_total_in_m2'] = propiedades['surface_total_in_m2'].apply(lambda x: np.nan if x <= 20 else x)
propiedades['surface_covered_in_m2'] = propiedades['surface_covered_in_m2'].apply(lambda x: np.nan if x <= 20 else x)
propiedades['surface_total_in_m2'] = propiedades['surface_total_in_m2'].apply(lambda x: np.nan if x >= 10000 else x)
propiedades['surface_covered_in_m2'] = propiedades['surface_covered_in_m2'].apply(lambda x: np.nan if x >= 1000 else x)
propiedades['floor'] = propiedades['floor'].apply(lambda x: np.nan if x >= 50 else x)
propiedades['rooms'] = propiedades['rooms'].apply(lambda x: np.nan if x >= 15 else x)

In [27]:
def validarExpensas(row):
    if(type(row['expenses']) == type(1) or type(row['expenses']) == type(1.)):
        return row['expenses']
    if row['expenses'].isdigit():
        return float(row['expenses'])
    return np.nan

In [28]:
propiedades['expenses'] = propiedades['expenses'].replace('[\$,)]','',  \
        regex=True).replace('[(]','-', regex=True)
propiedades['expenses'] = propiedades.apply(lambda row: validarExpensas(row), axis=1)

In [29]:
dic_categories['property_type'] = ['ph', 'departamento', 'casa']

In [30]:
for cat in categories:
    propiedades[cat] = propiedades[cat].apply(lambda x: categorical(x,cat))

In [31]:
propiedades = propiedades.drop(propiedades[cols_drop], axis=1)
CSV_NAME = 'data/testing.csv'
propiedades.to_csv(CSV_NAME, index=False)

In [32]:
df_tr = pd.read_csv(CSV_NAME, low_memory=False)
df_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 13 columns):
id                         14166 non-null int64
property_type              14166 non-null int64
place_name                 14166 non-null int64
place_with_parent_names    14166 non-null int64
state_name                 14166 non-null int64
lat                        10487 non-null float64
lon                        10487 non-null float64
surface_total_in_m2        11800 non-null float64
surface_covered_in_m2      12806 non-null float64
floor                      1357 non-null float64
rooms                      7495 non-null float64
expenses                   2514 non-null float64
description                14166 non-null object
dtypes: float64(7), int64(5), object(1)
memory usage: 1.4+ MB


In [ ]:
df_tr_grupos